In [ ]:
!pip install tf-quant-finance

In [4]:
import numpy as np
import tensorflow as tf
import tf_quant_finance as tff

In [6]:
# See https://github.com/google/tf-quant-finance/blob/2d0d7ac19944d0e1e3391c0e8c7d11940c969b11/tf_quant_finance/models/longstaff_schwartz/lsm.py#L147

# Let the underlying model be a Black-Scholes process
# dS_t / S_t = rate dt + sigma**2 dW_t, S_0 = 1.0
# with `rate = 0.1`, and volatility `sigma = 1.0`.
# Define drift and volatility functions for log(S_t)
rate = 0.1
def drift_fn(_, x):
    return rate - tf.ones_like(x) / 2.
def vol_fn(_, x):
    return tf.expand_dims(tf.ones_like(x), axis=-1)
# Use Euler scheme to propagate 100000 paths for 1 year into the future
times = np.linspace(0., 1, num=50)
num_samples = 100000
log_paths = tf.function(tff.models.euler_sampling.sample)(
      dim=1,
      drift_fn=drift_fn, volatility_fn=vol_fn,
      random_type=tff.math.random.RandomType.PSEUDO_ANTITHETIC,
      times=times, num_samples=num_samples, seed=42, time_step=0.01)
# Compute exponent to get samples of `S_t`
paths = tf.math.exp(log_paths)
# American put option price for strike 1.1 and expiry 1 (assuming actual day
# count convention and no settlement adjustment)
strike = [1.1]
exercise_times = tf.range(times.shape[-1])
discount_factors = tf.exp(-rate * times)
payoff_fn = tff.models.longstaff_schwartz.make_basket_put_payoff(strike)
basis_fn = tff.models.longstaff_schwartz.make_polynomial_basis(10)
print(tff.models.longstaff_schwartz.least_square_mc(paths, exercise_times, payoff_fn, basis_fn,
              discount_factors=discount_factors))
# Expected value: [0.397]

# European put option price
print(tff.black_scholes.option_price(volatilities=[1], strikes=strikes,
                             expiries=[1], spots=[1.],
                             discount_factors=discount_factors[-1],
                             is_call_options=False,
                             dtype=tf.float64))
# Expected value: [0.379]

ValueError: sample_paths: Tensor conversion requested dtype float32 for Tensor with dtype float64: <tf.Tensor: shape=(100000, 50, 1), dtype=float64, numpy=
array([[[1.        ],
        [1.00366749],
        [0.85731558],
        ...,
        [1.32189714],
        [1.46822627],
        [1.79168022]],

       [[1.        ],
        [1.02635853],
        [0.80834391],
        ...,
        [2.09295062],
        [1.87953774],
        [2.24183875]],

       [[1.        ],
        [0.94548095],
        [0.95919875],
        ...,
        [1.58425569],
        [2.11158356],
        [2.30087973]],

       ...,

       [[1.        ],
        [1.08738547],
        [1.25933449],
        ...,
        [1.25225301],
        [1.37185155],
        [1.19990301]],

       [[1.        ],
        [0.86710579],
        [0.87474619],
        ...,
        [0.41940581],
        [0.40283049],
        [0.42481206]],

       [[1.        ],
        [0.89370739],
        [0.87585211],
        ...,
        [2.987515